In [1]:
import random
import numpy as np
from generate import data

import warnings
warnings.filterwarnings("ignore")

### Non-sparse setting (Optional)

In [2]:
# 特征维度
p = 500
mean1 = np.repeat(0,p)
mean2 = np.repeat(1,p)
mean0 = np.repeat(0,p)
mean = [mean1,mean2,mean0]

# 方差矩阵列表
cov1 = np.diag(np.repeat(1,p))
cov2 = np.diag(np.repeat(1,p))
cov0 = np.diag(np.repeat(3.5,p))
cov = [cov1,cov2,cov0]

# 样本数量

n_train = 510
n_test = 2000

# 测试集中内点比例
purity = 2/3

# 是否添加均匀分布噪声
mixture = True

### Sparse Setting (Optional)

In [3]:
""" # 特征维度
p = 500
d = 250
mean1 = np.repeat(0,p)
mean2 = np.concatenate((np.repeat(1,d),np.repeat(0,p-d))) 
mean0 = np.concatenate((np.repeat(-1,d),np.repeat(0,p-d)))
mean = [mean1,mean2,mean0]

# 方差矩阵列表
cov1 = np.diag(np.repeat(1,p))
cov2 = np.diag(np.concatenate((np.array([1,1]),np.repeat(1, p-2))))
cov0 = np.diag(np.concatenate((np.repeat(1,d), np.repeat(1, p-d))))
for cov in [cov0,cov1,cov2]:
    for i in range(p):
        for j in range(p):
            if i!=j:
                cov[i,j] = 0.8
cov = [cov1,cov2,cov0]

# 样本数量

n_train = 510
n_test = 2000

# 测试集中内点比例
purity = 2/3

# 是否添加均匀分布噪声
mixture = True """

' # 特征维度\np = 500\nd = 250\nmean1 = np.repeat(0,p)\nmean2 = np.concatenate((np.repeat(1,d),np.repeat(0,p-d))) \nmean0 = np.concatenate((np.repeat(-1,d),np.repeat(0,p-d)))\nmean = [mean1,mean2,mean0]\n\n# 方差矩阵列表\ncov1 = np.diag(np.repeat(1,p))\ncov2 = np.diag(np.concatenate((np.array([1,1]),np.repeat(1, p-2))))\ncov0 = np.diag(np.concatenate((np.repeat(1,d), np.repeat(1, p-d))))\nfor cov in [cov0,cov1,cov2]:\n    for i in range(p):\n        for j in range(p):\n            if i!=j:\n                cov[i,j] = 0.8\ncov = [cov1,cov2,cov0]\n\n# 样本数量\n\nn_train = 510\nn_test = 2000\n\n# 测试集中内点比例\npurity = 2/3\n\n# 是否添加均匀分布噪声\nmixture = True '

### Classification

In [4]:
from RPCP import RPEnsemble_CP
from Base_classify import base
from Aggregate import AggregatedCP

In [5]:
n_exp = 5
exp_model = "KNN"
rp_name = "Gaussian"
alpha = 0.05
proj_d = 200

base_acc, base_rp_acc = [], []
rpecp_acc, rpecp_coverage, rpecp_length = [], [], [] 
icp_acc, icp_coverage, icp_length = [], [], []
icp_rp_acc, icp_rp_coverage, icp_rp_length = [], [], []
ccp_acc, ccp_coverage, ccp_length = [], [], [] 
ccp_rp_acc, ccp_rp_coverage, ccp_rp_length = [], [], [] 
bcp_acc, bcp_coverage, bcp_length = [], [], []
bcp_rp_acc, bcp_rp_coverage, bcp_rp_length = [], [], [] 

for exp in range(n_exp):
    print("exp:", exp+1, "start:")
    X = data(mean, cov, mixture = mixture)
    X_train = X.generate(n_train, purity, shuffle=False, random_state = 2024+exp)
    X_test = X.generate(n_test, purity, shuffle=True, random_state = 2024+exp)

    rpecp = RPEnsemble_CP(rp = rp_name, d = proj_d, B1=10, B2=50, classifier=exp_model, alpha=alpha)
    rpecp_res = rpecp.RP_Conformal(X_train, X_test)
    rpecp_acc.append(rpecp_res[0])
    rpecp_coverage.append(rpecp_res[1])
    rpecp_length.append(rpecp_res[2])
    print(rpecp_res)
    print("RPECP completed")

    base_classifier = base(classifier = exp_model)
    base_res = base_classifier.predict(X_train, X_test)
    base_acc.append(base_res)
    print(base_res)
    print("Base completed")

    base_rp_classifier = base(classifier = exp_model)
    base_rp_res = base_rp_classifier.predict(X_train, X_test, ran_d = proj_d, ran_m = rp_name)
    base_rp_acc.append(base_rp_res)
    print(base_rp_res)
    print("Base_RP completed")

    ICPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="ICP")
    icp_res = ICPredictor.predict(X_train, X_test)
    icp_acc.append(icp_res[0])
    icp_coverage.append(icp_res[1])
    icp_length.append(icp_res[2])
    print("ICP completed")

    ICPredictor_rp = AggregatedCP(classifier=exp_model, alpha=alpha, mode="ICP")
    icp_rp_res = ICPredictor_rp.predict(X_train, X_test, ran_d = proj_d, ran_m = rp_name)
    icp_rp_acc.append(icp_rp_res[0])
    icp_rp_coverage.append(icp_rp_res[1])
    icp_rp_length.append(icp_rp_res[2])
    print("ICP_RP completed")

    CCPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="CCP")
    ccp_res = CCPredictor.predict(X_train, X_test)
    ccp_acc.append(ccp_res[0])
    ccp_coverage.append(ccp_res[1])
    ccp_length.append(ccp_res[2])
    print("CCP completed")

    CCPredictor_rp = AggregatedCP(classifier=exp_model, alpha=alpha, mode="CCP")
    ccp_rp_res = CCPredictor_rp.predict(X_train, X_test, ran_d = proj_d, ran_m = rp_name)
    ccp_rp_acc.append(ccp_rp_res[0])
    ccp_rp_coverage.append(ccp_rp_res[1])
    ccp_rp_length.append(ccp_rp_res[2])
    print("CCP_RP completed")

    BCPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="BCP")
    bcp_res = BCPredictor.predict(X_train, X_test)
    bcp_acc.append(bcp_res[0])
    bcp_coverage.append(bcp_res[1])
    bcp_length.append(bcp_res[2])
    print("BCP_RP completed")
    
    BCPredictor_rp = AggregatedCP(classifier=exp_model, alpha=alpha, mode="BCP")
    bcp_rp_res = BCPredictor_rp.predict(X_train, X_test,ran_d = proj_d, ran_m = rp_name)
    bcp_rp_acc.append(bcp_rp_res[0])
    bcp_rp_coverage.append(bcp_rp_res[1])
    bcp_rp_length.append(bcp_rp_res[2])
    print("BCP completed")

    print("---------------------------------------\n")

exp: 1 start:
(0.79, 0.998, 1.6447895791583167)
RPECP completed
0.648
Base completed
0.443
Base_RP completed
ICP completed
ICP_RP completed
CCP with 2 folds
CCP completed
CCP with 2 folds
CCP_RP completed
BCP_RP completed
BCP completed
---------------------------------------

exp: 2 start:
(0.7765, 0.999, 1.66016016016016)
RPECP completed
0.644
Base completed
0.638
Base_RP completed
ICP completed
ICP_RP completed
CCP with 2 folds
CCP completed
CCP with 2 folds
CCP_RP completed
BCP_RP completed
BCP completed
---------------------------------------

exp: 3 start:
(0.7595, 1.0, 1.662)
RPECP completed
0.644
Base completed
0.6445
Base_RP completed
ICP completed
ICP_RP completed
CCP with 2 folds
CCP completed
CCP with 2 folds
CCP_RP completed
BCP_RP completed
BCP completed
---------------------------------------

exp: 4 start:
(0.7585, 0.997, 1.6725175526579739)
RPECP completed
0.6405
Base completed
0.6425
Base_RP completed
ICP completed
ICP_RP completed
CCP with 2 folds
CCP completed
CCP wi

In [6]:
print("base acc:", np.mean(base_acc), np.std(base_acc))
print("-----------------------------------------------------")
print("base_rp acc:", np.mean(base_rp_acc), np.std(base_rp_acc))
print("-----------------------------------------------------")
print("RPECP acc:", np.mean(rpecp_acc), np.std(rpecp_acc))
print("RPECP coverage:", np.mean(rpecp_coverage), np.std(rpecp_coverage))
print("RPECP length:", np.mean(rpecp_length), np.std(rpecp_length))
print("-----------------------------------------------------")
print("ICP acc:", np.mean(icp_acc), np.std(icp_acc))
print("ICP coverage:", np.mean(icp_coverage), np.std(icp_coverage))
print("ICP length:", np.mean(icp_length), np.std(icp_length))
print("-----------------------------------------------------")
print("ICP_RP acc:", np.mean(icp_rp_acc), np.std(icp_rp_acc))
print("ICP_RP coverage:", np.mean(icp_rp_coverage), np.std(icp_rp_coverage))
print("ICP_RP length:", np.mean(icp_rp_length), np.std(icp_rp_length))
print("-----------------------------------------------------")

base acc: 0.6447999999999999 0.0027313000567495416
-----------------------------------------------------
base_rp acc: 0.6028 0.07994535633793871
-----------------------------------------------------
RPECP acc: 0.7736000000000001 0.012666491226855236
RPECP coverage: 0.9984999999999999 0.0010000000000000009
RPECP length: 1.6599916056161212 0.008857902094725178
-----------------------------------------------------
ICP acc: 0.35150000000000003 0.00868907359849137
ICP coverage: 0.7352 0.004178516483155223
ICP length: 1.6061425696174247 0.018318225610876454
-----------------------------------------------------
ICP_RP acc: 0.3155 0.006480740698407867
ICP_RP coverage: 0.7384000000000001 0.005489990892524324
ICP_RP length: 1.6841098834182198 0.01475831660479196
-----------------------------------------------------


In [7]:
print("CCP acc:", np.mean(ccp_acc), np.std(ccp_acc))
print("CCP coverage:", np.mean(ccp_coverage), np.std(ccp_coverage))
print("CCP length:", np.mean(ccp_length), np.std(ccp_length))
print("-----------------------------------------------------")
print("CCP_RP acc:", np.mean(ccp_rp_acc), np.std(ccp_rp_acc))
print("CCP_RP coverage:", np.mean(ccp_rp_coverage), np.std(ccp_rp_coverage))
print("CCP_RP length:", np.mean(ccp_rp_length), np.std(ccp_rp_length))
print("-----------------------------------------------------")
print("BCP acc:", np.mean(bcp_acc), np.std(bcp_acc))
print("BCP coverage:", np.mean(bcp_coverage), np.std(bcp_coverage))
print("BCP length:", np.mean(bcp_length), np.std(bcp_length))
print("-----------------------------------------------------")
print("BCP_RP acc:", np.mean(bcp_rp_acc), np.std(bcp_rp_acc))
print("BCP_RP coverage:", np.mean(bcp_rp_coverage), np.std(bcp_rp_coverage))
print("BCP_RP length:", np.mean(bcp_rp_length), np.std(bcp_rp_length))

CCP acc: 0.25660000000000005 0.005616048432839593
CCP coverage: 0.7608999999999999 0.004115823125451329
CCP length: 1.8205550332823628 0.011997141357388687
-----------------------------------------------------
CCP_RP acc: 0.1777 0.0076589816555466435
CCP_RP coverage: 0.7739 0.005517245689653506
CCP_RP length: 2.0160269227809904 0.01106271088673144
-----------------------------------------------------
BCP acc: 0.3901 0.01281951637153291
BCP coverage: 0.7099 0.0038652296180175234
BCP length: 1.5004534529150355 0.019014708894883357
-----------------------------------------------------
BCP_RP acc: 0.3901 0.01281951637153291
BCP_RP coverage: 0.7099 0.0038652296180175234
BCP_RP length: 1.5004534529150355 0.019014708894883357
